In [1]:
from data import *

Using TensorFlow backend.


# Data Augmentasi

Pembuatan data augmentasi menggunakan library **keras.preprocessing.image.ImageDataGenerator** yang bisa menggabungkan Deep Neural Network image dan label secara bersamaan.

## Tentukan parameter data generator
Gunakan parameter dengan metode random dictionary seperti dibawah ini.

In [2]:
data_gen_args = dict(rotation_range = 0.05,
                    width_shift_range = 0.01,
                    height_shift_range = 0.01,
                    shear_range = 0.05,
                    zoom_range = 0.05,
                    horizontal_flip = True,
                    fill_mode = 'nearest')
myGenerator = trainGenerator(600, 'data/train', 'image', 'label', data_gen_args, save_to_dir = "data/train/aug")

## Membuat data augmentasi ke folder **'data/train/aug/'**

In [3]:
num_batch = 2
for i, batch in enumerate(myGenerator):
    print(i)
    if(i>=(num_batch-1)):
        break

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.
0
1


## Membuat .npy data (data augmentasi yang sudah dikompres)
Butuh memori tambahan untuk menyimpan data ini

In [4]:
image_arr,mask_arr = geneTrainNpy("data/train/aug/","data/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## Masking Test Data

In [37]:
def masking(img, mask, height=432, width=532, color = yellow):
    '''.'''
    #mask_out = np.zeros((512, 470, 3), dtype = 'uint8') #phantom
    mask_out = np.zeros((height, width, 3), dtype = 'uint8') #GE
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #np.zeros((height, width, 3), dtype = 'uint8') kalau shapenya (432,532) atau grayscale 
    for i in range(mask.shape[0]-1):
        for j in range(mask.shape[1]-1):
            if (mask[i,j] >= 30):
                mask_out[i,j,:] = mask[i,j] 
                mask_out[i,j,:] = color
                img[i,j,:] = mask_out[i,j,:]
    segmented = img
    return segmented

In [38]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

results_path = 'data/test/'
results_results = 'results/'
list_file = os.listdir(results_path)
masked = np.zeros((len(list_file)//2, 432, 532, 3), dtype ='uint8')

for i in range(len(list_file)//2):
    #masked[i] = masking(np.zeros((432, 532, 3), dtype = 'uint8'), io.imread(results_path+str(i)+'_predict.png'), color = white)
    masked[i] = masking(io.imread(results_path+str(i)+'.png'), io.imread(results_path+str(i)+'_predict.png'), color = yellow)
    if (i % 10 == 0):
        clear_output(wait=True)
    else:
        print('Creating... ' + results_path+str(i)+'_predict.png')

    io.imsave(os.path.join(results_results,"%d.png"%i), masked[i])

Creating... data/test/231_predict.png
Creating... data/test/232_predict.png
Creating... data/test/233_predict.png
Creating... data/test/234_predict.png
Creating... data/test/235_predict.png
Creating... data/test/236_predict.png
Creating... data/test/237_predict.png


# CARA CONVERT PNG TO MP4

In [ ]:
ffmpeg -r 30 -f image2 -i %d.png -vcodec libx264 -crf 15  -pix_fmt yuv420p test.mp4

# CARA MENGGABUNGKAN 2 VIDEO YANG SAMA

In [ ]:
ffmpeg -i Yoga60.avi -i aqua.mp4 -filter_complex "[0:v]pad=iw*2:ih[int]; [int][1:v]overlay=W/2:0[vid]" -map "[vid]" -c:v libx264 -crf 15 output2.mp4

## DICE SIMILARITY COEFFICIENT

In [28]:
def dice(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [58]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = dice(img1, img2)
    temp = temp + dsc[i]
    #print(dsc[i])
hasil = temp/len(list_predict)
print(hasil)

0.9349252204703714


## JACCARD SIMILARITY COEFFICIENT

In [59]:
import numpy as np


def jaccard(im1, im2):
    """
    Computes the Jaccard metric, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    jaccard : float
        Jaccard metric returned is a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
    
    Notes
    -----
    The order of inputs for `jaccard` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")
    
    intersection = np.logical_and(im1, im2)

    union = np.logical_or(im1, im2)

    return intersection.sum() / float(union.sum())

In [60]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = jaccard(img1, img2)
    temp = temp + dsc[i]
    #print(dsc[i])
hasil = temp/len(list_predict)
print(hasil)

0.8836660731418027


## F1-SCORE (Secara penurunan rumus sama seperti DSC)

In [1]:
import os
from os import listdir
from skimage import io
import cv2
from sklearn.metrics import f1_score
import numpy as np  

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
newf1 = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    ytrue = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY).ravel()/255
    ypred = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY).ravel()/255
    ytrue.astype(int)
    ypred.astype(int)
    newf1[i] = f1_score(ytrue, ypred, average= 'micro')
    temp = temp + newf1[i]
    #print(newf1[i])
hasil = temp/len(list_predict)
print(hasil)

0.9947097808421631


## PIXEL DIFFERENCE

In [33]:
import os
from os import listdir
from skimage import io
import cv2
import numpy as np 
import matplotlib.pyplot as plt 

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
temp = 0
aa = 0
bb = 0
for i in range(len(list_acuan)):
    a = np.count_nonzero(cv2.imread(path_predict+str(i)+'.png')==255)
    b = np.count_nonzero(cv2.imread(path_acuan+str(i)+'.png')==255)
    avg = abs(a-b)/b
    print(i, b, a, avg)
    temp = temp + avg
    aa = aa + a
    bb = bb + b
temp = temp/len(list_acuan)
aa = aa/len(list_acuan)
bb = bb/len(list_acuan)
print(aa, bb, temp)

0 8304 3642 0.5614161849710982
1 8913 3645 0.5910467855940761
2 8145 3642 0.552854511970534
3 7305 4347 0.40492813141683776
4 6519 4821 0.2604693971468017
5 6159 5268 0.1446663419386264
6 6144 5316 0.134765625
7 5949 5574 0.06303580433686334
8 5643 5541 0.018075491759702286
9 5679 5478 0.035393555203380875
10 5952 5493 0.07711693548387097
11 6147 5631 0.08394338701805759
12 6105 5871 0.03832923832923833
13 6486 6090 0.06105457909343201
14 6318 6078 0.03798670465337132
15 6666 6123 0.08145814581458145
16 6339 6129 0.03312825366777094
17 6552 6150 0.06135531135531135
18 6714 6024 0.10277033065236818
19 6525 5970 0.08505747126436781
20 6240 6117 0.01971153846153846
21 6327 5952 0.05926979611190138
22 6396 5667 0.11397748592870544
23 5823 5406 0.0716125708397733
24 5955 5319 0.10680100755667506
25 5937 5316 0.10459828196058615
26 5919 5280 0.10795742524075012
27 6135 5553 0.09486552567237164
28 6126 6348 0.03623898139079334
29 6057 6177 0.01981178801386825
30 5691 4722 0.17026884554559832


In [16]:
ytrue.max

<function ndarray.max>